In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder
from tqdm import tqdm
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.0.0+cu117


In [2]:
#from dataset import TerrainDataset
from datasetIMU import TerrainDatasetIMU
#from pointnet import PointNet
#from pointnetCurv import PointNetCurv

dataset = TerrainDatasetIMU(root='/home/atas/RESEARCH/traversablity_estimation_net/data_imu',train=True)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

from torch_geometric.loader import DataLoader
from pointnet_curv import PointnetCurv

train_dataset =  TerrainDatasetIMU(root='/home/atas/RESEARCH/traversablity_estimation_net/data_imu', train=True )
test_dataset = TerrainDatasetIMU(root='/home/atas/RESEARCH/traversablity_estimation_net/data_imu', train=False)

batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = PointnetCurv()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#criterion = torch.nn.MSELoss()  # Mean Squared Error Loss Function
criterion = torch.nn.L1Loss()  # Mean Absolute Error (L1 Loss Function)
#criterion = torch.nn.SmoothL1Loss()  # Huber Loss Function000000


# print number of model parameters
print("Number of model parameters: ", sum([p.numel() for p in model.parameters()]))

# print number of training samples
print("Number of training samples: ", len(train_dataset))
print("Number of test samples: ", len(test_dataset))

def train(model, optimizer, loader):
    model.train()
    total_loss = 0.0
    for data in loader:
        optimizer.zero_grad()                   # Clear gradients.
        # reshape to have 1 at the end
        data.pos = data.pos.reshape((data.pos.shape[0], data.pos.shape[1], 1))
        
        # make imu view as batch size x 13
        data.edge_attr = data.edge_attr.view(-1, 13)
        
        logits = model(data.pos, data.edge_attr, data.batch)                # Forward pass.
        loss = criterion(logits, data.y)        # Loss computation.
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss


@torch.no_grad()
def test(model, loader):
    model.eval()

    mae = 0.0
    for data in loader:
        data.pos = data.pos.reshape((data.pos.shape[0], data.pos.shape[1], 1))
        data.edge_attr = data.edge_attr.view(-1, 13)

        pred = model(data.pos, data.edge_attr, data.batch)
        
        mae += criterion(pred, data.y)        # Loss computation.

    mae = mae / len(loader)   
    # convert error to percentage accuracy
    return mae

<IPython.core.display.Javascript object>

Number of model parameters:  198414
Number of training samples:  632
Number of test samples:  84


In [4]:
train_loss=[]
test_mae=[]

# Save every 10th epoch model.

for epoch in range(1, 201):
    
    loss = train(model, optimizer, train_loader)
    mae = test(model, test_loader)
    if epoch % 50 == 0:
        torch.save(model.state_dict(), f'epoch_{epoch}.pt')
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, MAE: {mae:.4f}')
    train_loss.append(loss)
    test_mae.append(mae)
    
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 6), sharex=True)
ax1.plot(train_loss)
ax1.set_ylabel("training loss")
ax2.plot(test_mae)
ax2.set_ylabel("mae error")
ax2.set_xlabel("epochs")    

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 01, Loss: 166.3781, MAE: 0.1895
Epoch: 02, Loss: 135.4149, MAE: 0.2208
Epoch: 03, Loss: 129.1784, MAE: 0.1987
Epoch: 04, Loss: 123.0148, MAE: 0.2299


KeyboardInterrupt: 

In [ ]:
visual_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Load the model
# print curdir()
print(os.getcwd())

net = PointnetCurv()
net.load_state_dict(torch.load('/home/atas/RESEARCH/traversablity_estimation_net/scripts/epoch_100.pt'))
net.eval()

mae = 0.0
for data in visual_test_loader:
 
    inputs, labels = data.pos, data.y
    inputs = inputs.reshape((inputs.shape[0], inputs.shape[1], 1))
    outputs = net(inputs, data.x, data.batch)
    
    mae += criterion(outputs, data.y)        # Loss computation.
    
    outputs = outputs.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    
    print("Predicted: " + str(outputs) + " Actual: " + str(labels))

mae = mae / len(visual_test_loader)
print("MAE: " + str(mae))    
